In [192]:
import calendar
import os
import pandas as pd

# from exel_loader import df_operations, df_rigs, df_circ_data
# from db_loader import df_osi_operations, df_meters


# temp read from temp files

DATA_DIR = r'K:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/PYTHON/Scripts/OEE/python/data'

def read_csv(filename):
    path = os.path.join(DATA_DIR, filename)
    return pd.read_csv(path)

df_circ_data = read_csv('df_circ_data.csv')
df_meters = read_csv('df_meters.csv')
df_operations = read_csv('df_operations.csv')
df_osi_operations = read_csv('df_osi_operations.csv')
df_rigs = read_csv('df_rigs.csv')




# Функция для преобразования имён колонок:
def clean_column_names(columns):
    return [col.replace('_', ' ').title() for col in columns]

# Очищаем и переименовываем колонки сразу после чтения
df_circ_data.columns = clean_column_names(df_circ_data.columns)
df_meters.columns = clean_column_names(df_meters.columns)
df_operations.columns = clean_column_names(df_operations.columns)
df_osi_operations.columns = clean_column_names(df_osi_operations.columns)
df_rigs.columns = clean_column_names(df_rigs.columns)



In [193]:
# data modif 

def add_difference_column(df, col1, col2):
    """
    Возвращает Series: разница между двумя колонками с заменой NaN на 0.
    """
    return (df[col1].fillna(0) - df[col2].fillna(0)).clip(lower=0)


def add_ratio_column(df, numerator_col, denominator_col):
    """
    Возвращает Series: отношение numerator_col / denominator_col
    с защитой от деления на 0, заменой NaN на 0 и ограничением [0, 1].
    """
    num = df[numerator_col].fillna(0)
    denom = df[denominator_col].replace(0, pd.NA).fillna(pd.NA)
    ratio = num / denom
    return ratio.fillna(0).clip(0, 1)



# Работа с df_operations
df_operations = df_operations[['Event', 'Category']]
df_operations = df_operations[df_operations['Category'].notna()].reset_index(drop=True)


# Преобразуем Starttime и Endtime в datetime
df_osi_operations['Starttime'] = pd.to_datetime(df_osi_operations['Starttime'], errors='coerce')
df_osi_operations['Endtime'] = pd.to_datetime(df_osi_operations['Endtime'], errors='coerce')

# Добавляем колонку Duration в df_osi_operations
df_osi_operations['Duration'] = df_osi_operations['Endtime'] - df_osi_operations['Starttime']
df_osi_operations['Duration'] = df_osi_operations['Duration'].dt.total_seconds() / 3600

# Очищаем и фильтруем df_rigs
df_rigs = df_rigs[df_rigs['Rig-Acquire'].notna()].reset_index(drop=True)

# Приведение к одному формату для слияния
df_osi_operations['Equipment'] = df_osi_operations['Equipment'].str.strip().str.lower()
df_rigs['Rig-Osidem'] = df_rigs['Rig-Osidem'].str.strip().str.lower()


In [194]:
# Слияние
df_merged = df_osi_operations.merge(
    df_rigs,
    left_on='Equipment',
    right_on='Rig-Osidem',
    how='right'
)

df_temp = df_merged.drop(columns=['Equipment', 'Rig-Osidem'])

# Очистка колонок event
df_temp['Event'] = df_temp['Event'].str.strip().str.lower()
df_operations['Event'] = df_operations['Event'].str.strip().str.lower()

# Слияние с категориями
df_operations_total = df_temp.merge(df_operations, on='Event', how='left')
df_operations_total['Category'] = df_operations_total['Category'].fillna('Standard Work')

# Приведение endtime к datetime и фильтрация
df_operations_total['Endtime'] = pd.to_datetime(df_operations_total['Endtime'])
df_operations_total = df_operations_total.dropna(subset=['Endtime'])

In [195]:
# Добавляем year и month
df_operations_total['Year-Month'] = df_operations_total['Endtime'].dt.strftime('%Y-%m')

# Сводная таблица pivot
pivot_df = df_operations_total.pivot_table(
    index=['Drillcompany', 'Rig-Acquire', 'Year-Month', 'Tipe Of Circulation'],
    columns='Category',
    values='Duration',
    aggfunc='sum',
    fill_value=0
).reset_index()

In [196]:
pivot_df = pivot_df.sort_values(
    by=['Drillcompany', 'Rig-Acquire', 'Year-Month'],
    ascending=[True, True, True]
).reset_index(drop=True)

# Функция для подсчёта часов в месяце
def hours_in_month(row):
    year_month_str = row['Year-Month']  # предполагается, что у вас есть колонка с таким именем
    year, month = map(int, year_month_str.split('-'))
    days = calendar.monthrange(year, month)[1]
    return days * 24

In [197]:
pivot_df['H In Month'] = pivot_df.apply(hours_in_month, axis=1)

In [198]:
# Преобразование ENDDATE в datetime
df_meters['Enddate'] = pd.to_datetime(df_meters['Enddate'], errors='coerce')
df_meters.dropna(subset=['Enddate'], inplace=True)
df_meters['Year-Month'] = df_meters['Enddate'].dt.strftime('%Y-%m')


In [199]:
pivot2 = pd.pivot_table(
    df_meters,
    index=['Year-Month', 'Drillrig'],
    columns='Holestatus',
    values='Holeid',
    aggfunc='count',
    fill_value=0
)

In [200]:
depth_sum = df_meters.groupby(['Year-Month', 'Drillrig'])['Depth'].sum()
pivot2['Depth'] = depth_sum
meters_pivot = pivot2.reset_index()

# Приведение к верхнему регистру и очистка пробелов
pivot_df['Rig-Acquire'] = pivot_df['Rig-Acquire'].str.strip().str.upper()
meters_pivot['Drillrig'] = meters_pivot['Drillrig'].str.strip().str.upper()

total_merged_df = pd.merge(
    pivot_df,
    meters_pivot,
    left_on=['Rig-Acquire', 'Year-Month'],
    right_on=['Drillrig', 'Year-Month'],
    how='left'
)

In [201]:
total_merged_df.drop(columns='Rig-Acquire', inplace=True)

In [202]:
# Расчёты Planned Production Time и Planned Factor с ограничениями
total_merged_df['Planned Production Time'] = add_difference_column(total_merged_df, 'H In Month', 'Planned_downtime')

In [203]:
total_merged_df['Planned Factor'] = add_ratio_column(
    total_merged_df, 'Planned Production Time', 'H In Month'
)

# Gross Operating Time (GOT) и Availability
total_merged_df['Gross Operating Time'] = add_difference_column(
    total_merged_df, 'Planned Production Time', 'Unplanned_downtime_losses'
)

In [204]:
total_merged_df['Availability'] = add_ratio_column(
    total_merged_df, 'Gross Operating Time', 'Planned Production Time'
)

# Назначаем коэффициенты по типу циркуляции
circ_avg_drilling = df_circ_data.set_index('Circ')['Standard Avarage Drilling, M/H'].to_dict()
time_to_well_drill = df_circ_data.set_index('Circ')['Time To Well Drill, H'].to_dict()

# Записываем два столбца в df
total_merged_df['Circulation Coeff'] = total_merged_df['Tipe Of Circulation'].map(circ_avg_drilling)
total_merged_df['Well Drill Coef'] = total_merged_df['Tipe Of Circulation'].map(time_to_well_drill)


# Потенциальная глубина бурения
total_merged_df['Potential Depth'] = (
    total_merged_df['Gross Operating Time'] * total_merged_df['Circulation Coeff']
)

In [205]:
total_merged_df['Net Operating Time'] = total_merged_df['Depth'].div(
    total_merged_df['Circulation Coeff']
).fillna(0)

In [206]:
total_merged_df['Speed Losses'] = add_difference_column(
    total_merged_df, 'Gross Operating Time', 'Net Operating Time'
)

In [207]:
total_merged_df['Performance'] = add_ratio_column(total_merged_df, 'Net Operating Time', 'Gross Operating Time')

total_merged_df['Quality Losses'] =  total_merged_df.get('Well Drill Coef', 0) * total_merged_df.get('Liquid', 0)

# Valuable Operating Time
total_merged_df['Valuable Operating Time'] = add_difference_column(
    total_merged_df, 'Net Operating Time', 'Quality Losses'
    )


C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_13684\1726980654.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return ratio.fillna(0).clip(0, 1)


In [208]:
total_merged_df['Quality'] = add_ratio_column(
    total_merged_df, 'Valuable Operating Time', 'Net Operating Time'
    )

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_13684\1726980654.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return ratio.fillna(0).clip(0, 1)


In [209]:
total_merged_df.columns

Index(['Drillcompany', 'Year-Month', 'Tipe Of Circulation', 'Planned_downtime',
       'Standard Work', 'Unplanned_downtime_losses', 'H In Month', 'Drillrig',
       'ACCEPTED', 'LIQUID', 'NOT PROFITABLE', 'Depth',
       'Planned Production Time', 'Planned Factor', 'Gross Operating Time',
       'Availability', 'Circulation Coeff', 'Well Drill Coef',
       'Potential Depth', 'Net Operating Time', 'Speed Losses', 'Performance',
       'Quality Losses', 'Valuable Operating Time', 'Quality'],
      dtype='object')

In [210]:
total_merged_df['OEE'] = (
    total_merged_df['Availability'] *
    total_merged_df['Performance'] *
    total_merged_df['Quality']
)


total_merged_df['TRS'] = total_merged_df['OEE'] * total_merged_df['Planned Factor']

In [211]:
total_merged_df.columns = clean_column_names(total_merged_df.columns)

In [212]:
total_merged_df = total_merged_df.rename(columns={'Drillrig': 'Rig'})

In [213]:
df_losses = pd.melt(
    total_merged_df,
    id_vars=[
        'Drillcompany',
        'Rig',
        'Year-Month',
        'Tipe Of Circulation'
    ],
    value_vars=[
        'Planned Downtime',
        'Unplanned Downtime Losses',
        'Speed Losses',
        'Quality Losses'
    ],
    var_name='Loss Type',
    value_name='Loss Value'
)

In [214]:
df_productivity = pd.melt(
    total_merged_df,
    id_vars=[
        'Drillcompany',
        'Rig',
        'Year-Month',
        'Tipe Of Circulation'
    ],
    value_vars=[
        'Planned Factor',
        'Availability',
        'Performance',
        'Quality',
        'Oee',
        'Trs'
    ],
    var_name='Productivity Type',
    value_name='Productivity Value'
)

In [215]:
df_operations_total['Category'] = df_operations_total['Category'].replace({
    'Planned_downtime': 'Planned Downtime',
    'Unplanned_downtime_losses': 'Unplanned Downtime Losses'
})

In [216]:
df_operations_total = df_operations_total.rename(columns={'Rig-Acquire': 'Rig'})

In [217]:
df_operations_total['Event Category'] = df_operations_total['Event'].str.split('_').str[0].str.upper()
df_operations_total['Event'] = df_operations_total['Event'].str.split('_', n=1).str[1].str.capitalize()

In [218]:
df_events_duration = df_operations_total[
    [
        'Year-Month',
        "Category",
        'Drillcompany',
        'Rig',
        'Tipe Of Circulation',
        "Event Category",
        "Event",
        "Duration"
    ]
].sort_values(
    by=['Year-Month', 'Rig', 'Category', 'Duration'],
    ascending=[True, True, True, False],
)



In [219]:
df_meters.dropna(subset=['Drillcompany', 'Drillrig'], inplace=True)

In [220]:
df_meters.columns = [
    'Hole ID',
    'Drilling Company',
    'Rig',
    'Purpose',
    'Status',
    'End_Date',
    'Depth_m',
    'Year-Month'
]

df_meters = df_meters.drop(columns=['End_Date'])



In [221]:
print(df_meters)
print(df_events_duration)
print(df_productivity)
print(df_losses)


             Hole ID   Drilling Company             Rig  Purpose    Status  \
0     MSK39_08_09_A4       BurGeoProekt   PRAKLA_01-BGP  PRODUCT  ACCEPTED   
1     MSK03_06_02_A_       BurGeoProekt   PRAKLA_03-BGP  PRODUCT    LIQUID   
2     MSK03_04_05_A_       BurGeoProekt   PRAKLA_04-BGP  PRODUCT    LIQUID   
3     MSK03_04_09_A_       BurGeoProekt   PRAKLA_04-BGP  PRODUCT    LIQUID   
4     MSK04_05_04_A_       BurGeoProekt  ZIF1200_29-BGP   INJECT  ACCEPTED   
...              ...                ...             ...      ...       ...   
7096  TNU39_08_05_A_  TechnoService-Eng  ZIF1200_01-TSE  PRODUCT  ACCEPTED   
7097  TNU39_08_03_B_  TechnoService-Eng  ZIF1200_08-TSE  PRODUCT  ACCEPTED   
7098  TNU39_08_07_A_  TechnoService-Eng  ZIF1200_04-TSE  PRODUCT  ACCEPTED   
7099  TNU39_13_09_A_  TechnoService-Eng  ZIF1200_03-TSE   INJECT  ACCEPTED   
7100  TNU39_13_10_A_  TechnoService-Eng  ZIF1200_08-TSE   INJECT  ACCEPTED   

      Depth_m Year-Month  
0       508.0    2020-04  
1       4

In [222]:
unique_dates_series = pd.Series(
    sorted(df_productivity['Year-Month'].dropna().unique()),
    name='Year-Month'
)

unique_dates_series


0     2020-02
1     2020-03
2     2020-04
3     2020-08
4     2020-09
       ...   
57    2025-02
58    2025-03
59    2025-04
60    2025-05
61    2025-06
Name: Year-Month, Length: 62, dtype: object

In [ ]:
df_productivity['Productivity Type'].value_counts()

df_losses['Loss Type']

0        Planned Downtime
1        Planned Downtime
2        Planned Downtime
3        Planned Downtime
4        Planned Downtime
               ...       
11263      Quality Losses
11264      Quality Losses
11265      Quality Losses
11266      Quality Losses
11267      Quality Losses
Name: Loss Type, Length: 11268, dtype: object